In [1]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils import data
from d2l import torch as d2l
from tqdm import tqdm

In [13]:
reid_per = {}
reid_rec = {}

dict_per = pd.read_excel('./user.xlsx')
for i in dict_per.index:
  reid_per[i]=dict_per.loc[i,'求职者 ID']

dict_rec = pd.read_excel('./position.xlsx')
for i in dict_rec.index:
  reid_rec[i]=dict_rec.loc[i,'招聘信息 ID']


In [40]:
for i in range(10):
  print(np.random.random_sample())

0.4133108644642127
0.7856954562363295
0.5027749543624634
0.8404056975331617
0.47178969457126296
0.8930522532271762
0.04783286626162597
0.3104324210074513
0.663201429572203
0.6282585844341911


In [10]:
reid_per

{0: 1648621861036219904,
 1: 1648621300526210048,
 2: 1648345891960120064,
 3: 1648221000086710016,
 4: 1647944223779059968,
 5: 1647579226406249984,
 6: 1647012656068029952,
 7: 1646747130699840000,
 8: 1645422304206909952,
 9: 1646186991626350080,
 10: 1645750326164320000,
 11: 1645385086935360000,
 12: 1639549004641590016,
 13: 1627308899029870080,
 14: 1643560323032150016,
 15: 1641058737659699968,
 16: 1640319370922160128,
 17: 1639531141977469952,
 18: 1637801716407729920,
 19: 1637791761638220032,
 20: 1635610357617779968,
 21: 1634019092572790016,
 22: 1632947816114550016,
 23: 1632286586903330048,
 24: 1579833809761860096,
 25: 1631284316023680000,
 26: 1631491020585819904,
 27: 1631135062105910016,
 28: 1630852451164879872,
 29: 1631126636327990016,
 30: 1630105867208750080,
 31: 1630884821167369984,
 32: 1630818546172950016,
 33: 1623885034161299968,
 34: 1614575985309639936,
 35: 1599636587363300096,
 36: 1584881617820839936,
 37: 1529487074560960000,
 38: 15830247210317199

In [2]:
df_map = pd.read_csv('./per_job_map.csv')
df_map

,job,person,label
0,1135,40121,0
1,305,4973,0
2,664,9112,0
3,221,13978,0
4,322,41717,0
...,...,...,...
35286,1213,2949,0
35287,284,19247,0
35288,748,39496,0
35289,491,7187,1


In [3]:
# 读取属性嵌入
def get_info(path: str):
  info = pd.read_csv(path,sep=' ',header=None,na_values=[])
  info.drop(columns=[0,info.shape[1]-1],inplace=True)
  return info

info_job = get_info('./embedding_recruit.txt')
info_per = get_info('./embedding_person.txt')
info_job.shape, info_per.shape

((2104, 64), (54779, 219))

In [4]:
# 读取结构嵌入
def get_dest(path: str):
  dest = pd.read_csv(path,sep=' ',header=None)
  dest = dest.sort_values(0)
  dest.index = np.arange(dest.shape[0])
  return dest.drop(columns=0)
  
dest_job = get_dest('./destruct_job.txt')
dest_per = get_dest('./destruct_person.txt')
dest_job.shape, dest_per.shape

((2104, 128), (54779, 128))

In [5]:
# 拼接属性和结构，共539维
df_job_all = pd.concat([info_job, dest_job], axis=1)
df_per_all = pd.concat([info_per, dest_per], axis=1)
df_job_all.shape, df_per_all.shape

((2104, 192), (54779, 347))

In [8]:
df_job_all.to_csv('./all_job.csv')
df_job_all.to_excel('job_all.xlsx')
# df_per_all.to_csv('./all_person.csv')

In [6]:
def get_input(job_id, per_id):
  job = df_job_all.iloc[job_id-1].values
  per = df_per_all.iloc[per_id-1].values
  return np.concatenate([job, per],axis=0)

In [7]:
all_x = []
all_y = []
for i in tqdm(df_map.index):
  row = df_map.iloc[i]
  feature = get_input(row['job'],row['person'])
  if row['label']==0:
    label = [1,0.5]
  elif row['label']==1:
    label = [1,1]
  else:
    label = [0,0]
  label = np.array(label)
  all_x.append(feature)
  all_y.append(label)

100%|██████████| 35291/35291 [00:17<00:00, 1975.29it/s]


In [11]:
all_x = torch.Tensor(np.array(all_x))
all_y = torch.Tensor(np.array(all_y))
all_x.shape, all_y.shape
# all_x.shape

(torch.Size([35291, 539]), torch.Size([35291, 2]))

In [9]:
batch_size = 128
train_size = 30000
test_size = 5000

In [10]:
train_x_p = np.random.choice(all_x, train_size)
train_x_p

ValueError: a must be 1-dimensional

In [12]:
net = nn.Sequential(nn.Linear(539, 128),
                    nn.ReLU(),
                    nn.Linear(128, 2))
net

Sequential(
  (0): Linear(in_features=539, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=2, bias=True)
)

In [13]:
def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.01)

net.apply(init_weights)

Sequential(
  (0): Linear(in_features=539, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=2, bias=True)
)

In [14]:
def load_array(data_arrays, batch_size, is_train=True):
  # Array to Iter
  dataset = data.TensorDataset(*data_arrays)
  return data.DataLoader(dataset, batch_size, shuffle=is_train)

train_iter = load_array((all_x, all_y), batch_size, True)

In [15]:
lr, num_epochs = 0.01, 50
loss = nn.MSELoss()
trainer = torch.optim.SGD(net.parameters(), lr=lr)

In [16]:
num_epochs = 10
looper = tqdm(range(num_epochs))
for epoch in looper:
    for X, y in train_iter:
        # print(X.shape, y.shape)
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(all_x), all_y)
    # print(f'epoch {epoch + 1}, loss {l:f}')
    looper.set_description(f'epoch {epoch + 1}, loss {l:f}')
    

epoch 10, loss 0.016691: 100%|██████████| 10/10 [00:52<00:00,  5.28s/it]


In [5]:
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)